##  통계기반 기법

#### 말뭉치 또는 코퍼스(corpus)는 자연어 연구를 위해 특정한 목적을 가지고 언어의 표본을 추출한 집합이다.
#### 대량의 텍스트 데이터
컴퓨터의 발달로 말뭉치 분석이 용이해졌으며 분석의 정확성을 위해 해당 자연어를 형태소 분석하는 경우가 많다. 확률/통계적 기법과 시계열적인 접근으로 전체를 파악한다. 언어의 빈도와 분포를 확인할 수 있는 자료이며, 현대 언어학 연구에 필수적인 자료이다

In [1]:
text = 'You say goodbye and I say hello.' 

In [2]:
# 소문자로 변환
text = text.lower()
text

'you say goodbye and i say hello.'

In [3]:
text = text.replace('.',' .')
text

'you say goodbye and i say hello .'

In [4]:
words = text.split(' ')  
words # 단어 목록으로 변환

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

In [5]:
list(set(words)) # 중복된 단어를 제거해야할 경우

['.', 'i', 'you', 'hello', 'goodbye', 'and', 'say']

###  딕셔너리를 이용하여 단어 ID와 단어를 짝지어 주는 대응표 작성

In [13]:
word_to_id = {}   # dict , {'you':0, 'say':1,.....}
id_to_word = {}   # dict , {0:'you', 1:'say',.....}

for word in words: # 8회
    print(word)
    if word not in word_to_id:    # 중복 방지
        new_id = len(word_to_id)  # 0 ~ 6
        word_to_id[word] = new_id # word_to_id['you'] = 0
        id_to_word[new_id] = word # id_to_word[0] = 'you'

you
say
goodbye
and
i
say
hello
.


In [14]:
word_to_id

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}

In [15]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

In [16]:
word_to_id['hello']

5

In [18]:
id_to_word[2]

'goodbye'

### corpus를 숫자 벡터로 변환

In [19]:
# text = 'You say goodbye and I say hello.' 
# ['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']
# [  0,    1,       2,       3,     4,    1,    5,       6 ]  
import numpy as np
corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
print(corpus)

[0 1 2 3 4 1 5 6]


###  말뭉치를 이용하기 위한 전처리 함수 구현

In [20]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.',' .')
    words = text.split(' ')
    
    word_to_id = {}
    id_to_word = {}

    for word in words: 
        if word not in word_to_id:    
            new_id = len(word_to_id)  
            word_to_id[word] = new_id 
            id_to_word[new_id] = word 
            
    corpus = np.array([word_to_id[w] for w in words])
    
    return corpus, word_to_id, id_to_word

In [21]:
text = 'You say goodbye and I say hello.' 
corpus, word_to_id, id_to_word = preprocess(text)

In [22]:
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [23]:
word_to_id

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}

In [24]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

In [25]:
my_text = 'The cat ran very fast and the mouse could not run away.'
corpus, word_to_id, id_to_word = preprocess(my_text)
corpus

array([ 0,  1,  2,  3,  4,  5,  0,  6,  7,  8,  9, 10, 11])

In [26]:
word_to_id  # 12개 단어 사용

{'the': 0,
 'cat': 1,
 'ran': 2,
 'very': 3,
 'fast': 4,
 'and': 5,
 'mouse': 6,
 'could': 7,
 'not': 8,
 'run': 9,
 'away': 10,
 '.': 11}

In [27]:
id_to_word

{0: 'the',
 1: 'cat',
 2: 'ran',
 3: 'very',
 4: 'fast',
 5: 'and',
 6: 'mouse',
 7: 'could',
 8: 'not',
 9: 'run',
 10: 'away',
 11: '.'}

### 분포가설(distibutional hypothesis) : 단어의 의미는 주변 단어에 의해 형성된다

### 동시 발생 행렬(Co-occurence Matrix)
: 모든 단어의 동시 발생 횟수를 벡터로 표현한 행렬

- 주변 단어(맥락,Contexts)
- 중간 단어(타깃,target)
- 윈도 사이즈 : 중간단어를 기준으로 앞뒤로 사용할 주변 단어 갯수

In [28]:
text = 'You say goodbye and I say hello.' 
corpus, word_to_id, id_to_word = preprocess(text)

In [29]:
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [30]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

In [31]:
C = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 1, 1, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 1, 0],
], dtype=np.int32)

In [32]:
C

array([[0, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0]])

In [33]:
C[0] # id 가 0인 단어('you')의 벡터 표현

array([0, 1, 0, 0, 0, 0, 0])

In [34]:
C[4] # id 가 4인 단어('i')의 벡터 표현

array([0, 1, 0, 1, 0, 0, 0])

In [ ]:
# 동시 발생 행렬을 생성하는 함수 구현